<a href="https://colab.research.google.com/github/efearase/RL_with_sentiment/blob/main/features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from moexalgo import Ticker
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def calculate_RSI(data, period=14):
    delta = data.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))

    return RSI

def calculate_MACD(data, short_period=12, long_period=26, signal_period=9):
    short_EMA = data.ewm(span=short_period, adjust=False).mean()
    long_EMA = data.ewm(span=long_period, adjust=False).mean()

    MACD = short_EMA - long_EMA
    signal_line = MACD.ewm(span=signal_period, adjust=False).mean()
    histogram = MACD - signal_line

    return MACD

# Assuming df['Close'] contains the close prices
# df['RSI'] = calculate_RSI(df['Close'])
# df['MACD'], df['Signal_Line'], df['Histogram'] = calculate_MACD(df['Close'])

In [ ]:
names = pd.read_csv('Export_ru_securities-list_20240129.csv', encoding='windows-1251', delimiter=';', quotechar='"')
codes = names[((names['LIST_SECTION']=='Первый уровень')) & (names['SUPERTYPE']=='Акции')]['TRADE_CODE'].dropna()

df_list = []

for code in codes:
    data = Ticker(code).candles(date='2022-03-20', till_date='today', period='1D')

    data['RSI'] = calculate_RSI(data['close'])
    data['MACD']= calculate_MACD(data['close'])

    data['Symbols'] = code

    data.rename(columns={'close': 'Adj Close', 'high': 'High', 'low': 'Low', 'open': 'Open', 'volume': 'Volume', 'begin':'Date'}
                , inplace=True)

    df_list.append(data)

panel_data = pd.concat(df_list, ignore_index=True)

panel_data.sort_values(by='Date', inplace=True)
# panel_data.set_index('Date', inplace=True)
panel_data.to_csv('stocks_data/stocks_panel_data.csv', index=False)

In [ ]:
panel_data.drop(columns = ['value', 'end'], inplace=True)
panel_data

,Open,Adj Close,High,Low,Volume,Date,RSI,MACD,Symbols
0,5.100,5.534,5.663,5.10,33721900.0,2022-03-24,NaN,0.000000,CBOM
24749,312.000,360.200,375.000,308.00,18407648.0,2022-03-24,NaN,0.000000,ROSN
12226,94.510,100.750,110.100,91.11,7931080.0,2022-03-24,NaN,0.000000,MOEX
5785,11.505,12.890,14.414,11.50,93857400.0,2022-03-24,NaN,0.000000,AFKS
12717,5302.500,5525.000,5993.000,5300.00,1203273.0,2022-03-24,NaN,0.000000,LKOH
...,...,...,...,...,...,...,...,...,...
8726,2255.000,2273.000,2298.000,2235.20,135790.0,2024-02-29,58.018018,29.524127,POSI
8237,1010.000,975.000,1027.800,971.40,3148762.0,2024-02-29,76.711644,57.532646,LSRG
24257,3599.000,3708.500,3740.000,3595.00,457466.0,2024-02-29,49.423520,-53.426846,SMLT
13696,5629.000,5660.000,5660.000,5601.00,40899.0,2024-02-29,48.083624,2.757702,BELU


In [ ]:
data

,Open,Adj Close,High,Low,value,Volume,Date,end,RSI,MACD,Symbols
0,313.0,329.0,369.9,300.4,604524109.4,1767605.0,2022-03-24,2022-03-24 23:59:59,NaN,0.000000,TATNP
1,349.0,318.5,349.0,315.0,194851660.0,605574.0,2022-03-25,2022-03-25 23:59:59,NaN,-0.837607,TATNP
2,326.0,318.0,327.0,303.0,178589322.5,569398.0,2022-03-28,2022-03-28 23:59:59,NaN,-1.524192,TATNP
3,321.8,336.7,344.0,313.0,397585228.9,1188152.0,2022-03-29,2022-03-29 23:59:59,NaN,-0.553008,TATNP
4,336.9,351.8,356.4,330.1,314080759.7,894241.0,2022-03-30,2022-03-30 23:59:59,NaN,1.418753,TATNP
...,...,...,...,...,...,...,...,...,...,...,...
486,682.8,690.9,695.8,680.5,162380263.5,236475.0,2024-02-22,2024-02-22 23:59:59,35.619048,0.758325,TATNP
487,693.6,698.6,702.7,692.7,236081387.8,338194.0,2024-02-26,2024-02-26 23:59:59,42.123288,0.690293,TATNP
488,698.6,691.2,699.0,689.0,217413151.9,313477.0,2024-02-27,2024-02-27 23:59:59,36.024845,0.038811,TATNP
489,691.1,694.8,698.8,689.9,173374753.4,249183.0,2024-02-28,2024-02-28 23:59:59,36.419753,-0.184872,TATNP
